In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2

from jupyterthemes import jtplot
jtplot.style()

Populating the interactive namespace from numpy and matplotlib


In [2]:
import os, sys, time
import cPickle as pickle
sys.path.append('../EXP/')
import ClevelandMcGill as C
reload(C)

<module 'ClevelandMcGill' from '../EXP/ClevelandMcGill/__init__.pyc'>

In [3]:
N = 100000
images = np.zeros((N,100,100), dtype=np.bool)
labels = np.zeros((N), dtype=np.float32)

for n in range(N):
  
  sparse, image, label, parameters = C.Figure1.angle([True, True])
  
  images[n] = image
  labels[n] = label
  

In [14]:
X = images.astype(np.float32) - .5
y = (labels-np.min(labels))/(np.max(labels)-np.min(labels))
print 'memory usage', (X.nbytes + y.nbytes) / 1000000., 'MB'

memory usage 4000.4 MB


In [19]:
X3D = np.stack((X,)*3, -1)
print 'memory usage', (X3D.nbytes + y.nbytes) / 1000000., 'MB'

memory usage 12000.4 MB


In [20]:
import keras.applications

VGG19 = keras.applications.VGG19(include_top=False, weights='imagenet', input_shape=(100,100,3))

t0 = time.time()
features = VGG19.predict(X3D, verbose=True)
print 'VGG19 features done after', time.time()-t0

100000/100000 [==============================] - 281s 3ms/step
VGG19 features done after 280.728364944


In [22]:
oshape = features.shape
print 'features shape', oshape
print 'memory usage', (features.nbytes + y.nbytes) / 1000000., 'MB'

features shape (100000, 3, 3, 512)
memory usage 1843.6 MB


In [24]:
from keras import models
from keras import layers
from keras import optimizers
import keras.applications
from keras import backend as K
from keras.utils.np_utils import to_categorical

MLP = models.Sequential()

MLP.add(layers.Dense(256, activation='relu', input_dim=oshape[1]*oshape[2]*oshape[3]))
MLP.add(layers.Dropout(0.5))
MLP.add(layers.Dense(1, activation='linear')) # REGRESSION

sgd = optimizers.SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
MLP.compile(loss='mean_squared_error', optimizer=sgd, metrics=['mse', 'mae']) # MSE for regression

In [45]:
t0 = time.time()
callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='auto')]

history = MLP.fit(features[0:60000].reshape(60000, oshape[1]*oshape[2]*oshape[3]), \
                  y[0:60000], \
                  epochs=1000, \
                  batch_size=32, \
                  validation_split=0.25,
                  callbacks=callbacks,
                  verbose=True)
print 'Fitting done', time.time()-t0

Train on 45000 samples, validate on 15000 samples
Epoch 1/1000
45000/45000 [==============================] - 7s 146us/step - loss: 0.0136 - mean_squared_error: 0.0136 - mean_absolute_error: 0.0898 - val_loss: 0.0091 - val_mean_squared_error: 0.0091 - val_mean_absolute_error: 0.0728
Epoch 2/1000
45000/45000 [==============================] - 7s 145us/step - loss: 0.0134 - mean_squared_error: 0.0134 - mean_absolute_error: 0.0890 - val_loss: 0.0090 - val_mean_squared_error: 0.0090 - val_mean_absolute_error: 0.0716
Epoch 3/1000
45000/45000 [==============================] - 7s 146us/step - loss: 0.0133 - mean_squared_error: 0.0133 - mean_absolute_error: 0.0889 - val_loss: 0.0086 - val_mean_squared_error: 0.0086 - val_mean_absolute_error: 0.0699
Epoch 4/1000
45000/45000 [==============================] - 7s 146us/step - loss: 0.0132 - mean_squared_error: 0.0132 - mean_absolute_error: 0.0882 - val_loss: 0.0086 - val_mean_squared_error: 0.0086 - val_mean_absolute_error: 0.0705
Epoch 5/1000
4

Epoch 36/1000
45000/45000 [==============================] - 7s 146us/step - loss: 0.0107 - mean_squared_error: 0.0107 - mean_absolute_error: 0.0794 - val_loss: 0.0067 - val_mean_squared_error: 0.0067 - val_mean_absolute_error: 0.0618
Epoch 37/1000
45000/45000 [==============================] - 7s 145us/step - loss: 0.0105 - mean_squared_error: 0.0105 - mean_absolute_error: 0.0789 - val_loss: 0.0064 - val_mean_squared_error: 0.0064 - val_mean_absolute_error: 0.0601
Epoch 38/1000
45000/45000 [==============================] - 7s 147us/step - loss: 0.0104 - mean_squared_error: 0.0104 - mean_absolute_error: 0.0784 - val_loss: 0.0070 - val_mean_squared_error: 0.0070 - val_mean_absolute_error: 0.0629
Epoch 39/1000
45000/45000 [==============================] - 7s 146us/step - loss: 0.0103 - mean_squared_error: 0.0103 - mean_absolute_error: 0.0780 - val_loss: 0.0064 - val_mean_squared_error: 0.0064 - val_mean_absolute_error: 0.0605
Epoch 40/1000
45000/45000 [==============================] -

Epoch 71/1000
45000/45000 [==============================] - 7s 145us/step - loss: 0.0092 - mean_squared_error: 0.0092 - mean_absolute_error: 0.0735 - val_loss: 0.0059 - val_mean_squared_error: 0.0059 - val_mean_absolute_error: 0.0575
Epoch 72/1000
45000/45000 [==============================] - 6s 144us/step - loss: 0.0091 - mean_squared_error: 0.0091 - mean_absolute_error: 0.0733 - val_loss: 0.0055 - val_mean_squared_error: 0.0055 - val_mean_absolute_error: 0.0558
Epoch 73/1000
45000/45000 [==============================] - 6s 144us/step - loss: 0.0091 - mean_squared_error: 0.0091 - mean_absolute_error: 0.0731 - val_loss: 0.0054 - val_mean_squared_error: 0.0054 - val_mean_absolute_error: 0.0552
Epoch 74/1000
45000/45000 [==============================] - 7s 146us/step - loss: 0.0090 - mean_squared_error: 0.0090 - mean_absolute_error: 0.0730 - val_loss: 0.0057 - val_mean_squared_error: 0.0057 - val_mean_absolute_error: 0.0570
Epoch 75/1000
45000/45000 [==============================] -

Epoch 106/1000
45000/45000 [==============================] - 7s 145us/step - loss: 0.0085 - mean_squared_error: 0.0085 - mean_absolute_error: 0.0703 - val_loss: 0.0047 - val_mean_squared_error: 0.0047 - val_mean_absolute_error: 0.0519
Epoch 107/1000
45000/45000 [==============================] - 7s 145us/step - loss: 0.0084 - mean_squared_error: 0.0084 - mean_absolute_error: 0.0700 - val_loss: 0.0049 - val_mean_squared_error: 0.0049 - val_mean_absolute_error: 0.0524
Fitting done 701.590909958


In [46]:
y_pred = MLP.predict(features[60000:].reshape(40000, oshape[1]*oshape[2]*oshape[3]))

In [48]:
import sklearn.metrics
y_test = y[60000:]
np.log2(sklearn.metrics.mean_absolute_error(y_pred*100, y_test*100)+.125)

2.4146402076526385